In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_n


In [7]:
from enum import Enum
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import OutputFixingParser

# 1. Define the Enum for sentiment categories
class Sentiment(Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# 2. Set up the Output Parser
output_parser = EnumOutputParser(enum=Sentiment)

# Optional: Add an OutputFixingParser for robustness
fix_parser = OutputFixingParser.from_llm(parser=output_parser, llm=ChatOpenAI(model="moonshotai/kimi-k2-instruct-0905"))

# 3. Define the Prompt Template
prompt_template = PromptTemplate(
    template="""주어진 영화 리뷰의 감정을 '긍정', '부정', '보통' 중 하나로 분류하세요.
출력은 반드시 한국어로 된 단어 하나여야 합니다.

리뷰: {review}
감정: """,
    input_variables=["review"],
)

# 4. Initialize the LLM
llm =  ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 5. Create the Chain
# We use the fixing parser to make the chain more robust
chain = prompt_template | llm | fix_parser

# 6. Test Reviews
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    "스토리가 너무 복잡해서 이해하기 어려웠지만, 시각 효과는 뛰어났습니다."
]

# 7. Run the tests and print results
print("--- 영화 리뷰 감정 분석 ---")
for review in test_reviews:
    print(f"리뷰: \"{review}\"")
    result = chain.invoke({"review": review})
    
    # The parser returns the Enum member, we can access its value
    if isinstance(result, Sentiment):
        sentiment_label = result.value
        print(f"감정: {sentiment_label}")
    else:
        print(f"감정: {result}")
        
    print("-" * 20)

--- 영화 리뷰 감정 분석 ---
리뷰: "이 영화 정말 재미없어요. 시간 낭비였습니다."
감정: 부정
--------------------
리뷰: "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
감정: 긍정
--------------------
리뷰: "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
감정: 보통
--------------------
리뷰: "스토리가 너무 복잡해서 이해하기 어려웠지만, 시각 효과는 뛰어났습니다."
감정: 보통
--------------------
